<a href="https://colab.research.google.com/github/davisdw/Lending_Tree_Loan_Prediction_Analysis/blob/main/pyspark_data_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'

import os

spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
%pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [50.4 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,522 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe 

In [24]:
# Mounted Google Drive to access Data Source *DO NOT RERUN*
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pyspark
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
from pyspark import SparkFiles

url = "/content/drive/MyDrive/Lending_Club_Data_Source/accepted_2007_to_2018Q4.csv"

spark.sparkContext.addFile(url)
loan_accept_df = spark.read.csv(SparkFiles.get("accepted_2007_to_2018Q4.csv"), sep=",", header=True)

# Display the Data
loan_accept_df.show()

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-

In [6]:
# Create Temp View Tables
loan_accept_df.createOrReplaceTempView("accepted")

In [8]:
# Truncate un-needed columns from the "accepted" table
# Tables that we're keeping -->>:loan_amnt, int_rate, emp_title, emp_length,
# home_ownership, annual_inc, issue_d, loan_status, purpose, addr_state, dti,fico_range_low, fico_range_high

a = """ SELECT
        loan_amnt,
        int_rate,
        emp_title,
        emp_length,
        home_ownership,
        annual_inc,
        issue_d,
        loan_status,
        purpose,
        addr_state,
        dti,fico_range_low,
        fico_range_high
        FROM accepted
    """

spark.sql(a).show()


+---------+--------+--------------------+----------+--------------+----------+--------+-----------+------------------+----------+-----+--------------+---------------+
|loan_amnt|int_rate|           emp_title|emp_length|home_ownership|annual_inc| issue_d|loan_status|           purpose|addr_state|  dti|fico_range_low|fico_range_high|
+---------+--------+--------------------+----------+--------------+----------+--------+-----------+------------------+----------+-----+--------------+---------------+
|   3600.0|   13.99|             leadman| 10+ years|      MORTGAGE|   55000.0|Dec-2015| Fully Paid|debt_consolidation|        PA| 5.91|         675.0|          679.0|
|  24700.0|   11.99|            Engineer| 10+ years|      MORTGAGE|   65000.0|Dec-2015| Fully Paid|    small_business|        SD|16.06|         715.0|          719.0|
|  20000.0|   10.78|        truck driver| 10+ years|      MORTGAGE|   63000.0|Dec-2015| Fully Paid|  home_improvement|        IL|10.78|         695.0|          699.0

In [13]:
# Cache the accepted dataset table
spark.sql("cache table accepted")

DataFrame[]

In [15]:
# Partitioning into saller datasets and output the data into multiple folders
loan_accept_df.write.partitionBy("issue_d").mode("overwrite").csv("accepted_parquet_csv")


In [20]:
#Test by reading the partioned data
readPartitionData = spark.read.option("header", True)\
.csv("/content/accepted_parquet_csv/issue_d=Apr-2008/part-00009-1aa3a6d7-521e-4f91-a958-119306660558.c000.csv")
readPartitionData.printSchema()
readPartitionData.show()

root
 |-- 345705: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- 7500.02: string (nullable = true)
 |-- 7500.03: string (nullable = true)
 |-- 214.02: string (nullable = true)
 |-- 36 months: string (nullable = true)
 |-- 8.0: string (nullable = true)
 |-- 235.03: string (nullable = true)
 |-- A: string (nullable = true)
 |-- A3: string (nullable = true)
 |-- JEK: string (nullable = true)
 |-- 7 years: string (nullable = true)
 |-- MORTGAGE: string (nullable = true)
 |-- 45000.0: string (nullable = true)
 |-- Not Verified: string (nullable = true)
 |-- Fully Paid: string (nullable = true)
 |-- n16: string (nullable = true)
 |-- https://lendingclub.com/browse/loanDetail.action?loan_id=345705: string (nullable = true)
 |-- Loan is for working capital for starting up a pawn shop.   20% of the United states population does not have a bank.  Lending and credit services are provided to individuals who do not have cash resources or access to credit to meet short term cash ne

In [18]:
# Read the entire csv partition collectively
partition_accepted_df = spark.read.format("csv").option("header", True).load("/content/accepted_parquet_csv")

In [19]:
# Display the partitions csv
partition_accepted_df.show()

+--------+----+--------+--------+--------+---------+-----+-------+---+---+--------------------+---------+--------+--------+---------------+-----------------+---+-----------------------------------------------------------------+----+------------------+--------------------+-----+---+-----+-----+--------+-----+-----+-----+----+-----+----+-----+--------+------+----+---+--------+--------+------------------+----------+--------+--------+-----+-------+------------------+--------+--------+--------+--------+-----+-----+-----+----+-----+----------+----+----+----+-----+------+--------+-----+-----+-----+-----+----+--------+----+-----+-----+-------+------+--------+-----+-----+-----+----+-------+--------+----+-----+-----+-----+-----+-----+-----+---+-----+----+------+----+-----+----+------+------+------+-----+------+----+----+----+-----+------+------+------+-----+-----+------+------+--------+--------+--------+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-

In [21]:
#Un-cached the accepted table
spark.sql("uncache table accepted")

DataFrame[]

In [22]:
# Check to see if the table is cached
if spark.catalog.isCached('accepted'):
  print('accepted dataset table is cached')
else:
  print('accepted dataset table is not there')

accepted dataset table is not there


Cache the rejected temp table and partitioned the data into CSV

In [25]:
# Read in the loan rejection data csv

url_2 = "/content/drive/MyDrive/Lending_Club_Data_Source/rejected_2007_to_2018Q4.csv"

spark.sparkContext.addFile(url_2)
loan_reject_df = spark.read.csv(SparkFiles.get("rejected_2007_to_2018Q4.csv"), sep=',', header=True)

loan_reject_df.show()

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|Application Date|          Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|                 10%|   481xx|   NM|          4 years|        0.0|
|          1000.0|      2007-05-26|  Consolidating Debt|     703.0|                 10%|   010xx|   MA|         < 1 year|        0.0|
|         11000.0|      2007-05-27|Want to consolida...|     715.0|                 10%|   212xx|   MD|           1 year|        0.0|
|          6000.0|      2007-05-27|             waksman|     698.0|              38.64%|   017xx|   MA|         < 1 year|        0.0|
|          1500.0|      2007-05-27|              mdrigo|     5

In [26]:
loan_reject_df.createOrReplaceTempView("rejected")

In [28]:
# Truncate un-needed columns from the "rejected" table
# Tables that we're keeping -->>:

b = """ SELECT
        Risk_Score
        FROM rejected
    """

spark.sql(b).show()

+----------+
|Risk_Score|
+----------+
|     693.0|
|     703.0|
|     715.0|
|     698.0|
|     509.0|
|     645.0|
|     693.0|
|     700.0|
|     694.0|
|     573.0|
|     710.0|
|     680.0|
|     688.0|
|     704.0|
|     694.0|
|     708.0|
|     685.0|
|     698.0|
|     680.0|
|     680.0|
+----------+
only showing top 20 rows



In [ ]:
# Cache the Rejected dataset table
spark.sql("cache table rejected")

In [ ]:
loan_reject_df.write.partitionBy("Application Date").mode("overwrite").csv("rejected_parquet_csv")

In [ ]:
# Converting them over to Pandas DataFrame
loan_accept_df.toPandas()
loan_accept_df.toPandas()

In [ ]:

spark.sql("uncache table rejected")